This project is licensed under the terms of the Creative Commons CC BY-NC-ND 4.0 license.

# Artificial Dissipation

If the resistive term is not included in the magnetic induction equation and the energy
preserving semidiscretisations are used, there is basically no
dissipation, except at boundaries due to the imposition of boundary conditions.
Thus, the varying velocity $u$ can cause some problems.
Moreover, instabilities and unphysical oscillations will occur at least at discontinuities.
If a full plasma model is considered, the nonlinearity allows discontinuities in the
velocity $u$ and the magnetic field $B$, even if smooth initial and boundary conditions
are given. Thus, some kind of dissipation is necessary in general. Even if the resistive
term $- \nabla \times (\eta \nabla \times B)$ is used explicitly, the discrete grid
might not suffice to resolve all scales of the physical dissipation. Thus, nonlinear
instabilities can still trigger oscillations and problems at steep gradients such
as the well-known Gibbs phenomenon. Therefore, artificial dissipation will be used.

[Mattsson, Svärd and Nordström (2004)](#mattsson2004stable) proposed high order artificial 
dissipation operators for SBP derivative operators in order to damp high frequency noise.
Based on these results, shock capturing artificial dissipation methods have been
developed by [Svärd and Mishra (2009)](#svard2009shock).
Since some adaptations of these schemes have been implemented in this repository, a brief
description of these methods is given in the following. Since the tensor product
structure will be used for schemes in multiple space dimensions, the following
description is given in one space dimension.

The artificial dissipation operators of order $2p$ developed by 
[Mattsson, Svärd and Nordström (2004)](#mattsson2004stable) are of the form

\begin{equation}
\tag{1}
  A_{2p}
  =
  - \Delta x^{2p+1} M^{-1} D_p^T a_{2p} D_p,
\end{equation}

where $M$ is the mass/norm matrix of the SBP operator, $D_p$ an
approximation of the $p$-th derivative with minimal band width, and $a_{2p}$
is a diagonal mass matrix with non-negative entries. The scaling with $\Delta x^{2p+1}$
cancels all divisions by $\Delta x$ on the following terms and makes this operator
$2p$-th order accurate, since it is basically an undivided difference operator
approximating the $2p$-th derivative. $a_{2p}$ vanishes at the boundary and
is of order $\mathcal{O}(1)$ in the interior.

#### <a id="ex:SBP-2-HO-AD"></a>Example 1

>  For the classical second order SBP operator
>
> \begin{equation}
    D
    =
    \frac{1}{2 \Delta x}
    \begin{pmatrix}
      -2 & 2 \\
      -1 & 0 & 1 \\
      & \ddots & \ddots & \ddots \\
      && -1 & 0 & 1 \\
      &&& -2 & 2
    \end{pmatrix},
    \qquad
    M
    =
    \Delta x
    \begin{pmatrix}
      \frac{1}{2} \\
      & 1 \\
      && \ddots \\
      &&& 1 \\
      &&&& \frac{1}{2}
    \end{pmatrix},
  \end{equation}
>
>  the corresponding dissipation operator of the form (1) is given by ${a_2}_1 = 0$
>  and
>
> \begin{equation}
  \begin{gathered}
    \left( {A_2} {B} \right)_1
    =
    2 {a_2}_2 ({B}_2 - {B}_1),
    \quad
    \left( {A_2} {B} \right)_N
    =
    - 2 {a_2}_N ({B}_N - {B}_{N-1}),
    \\
    \left( {A_2} {B} \right)_c
    =
      {a_2}_{c+1} ({B}_{c+1} - {B}_c)
    - {a_2}_{c} ({B}_{c} - {B}_{c-1}), \; c \in \{ 2, \dots, N-1 \}.
  \end{gathered}
  \end{equation}
>
>  If nothing else is noted explicitly, the other coefficients of ${a_2}$
>  are set to one.


The shock capturing artificial dissipation operators of [Svärd and Mishra (2009)](#svard2009shock)
are similar to (1). However, instead of a discrete version $D_p$
of $\partial_x^p$, another difference operator is used. Moreover, the weights are
chosen differently. Thus, the formal order of accuracy is in general not $2p$.
Using the representation of higher order central difference approximations as
linear combinations of second order central differences, linear combinations with
positive coefficients of artificial dissipation operators of the form given in
Example 1 are applied.

Basically, the weight $a_c$ of an artificial dissipation term
$a_c (B_{c+1} - B_{c})$ is chosen as

\begin{equation}
\tag{2}
  a_c
  =
  \frac{1}{2 \Delta x} \sigma(B_{c+1}, B_{c}),
\end{equation}

where $\sigma(B_{c+1}, B_{c})$ is the dissipation coefficient. Note
especially the scaling with $\Delta x^{-1}$, resulting in a first order method.
In the following,

\begin{equation}
\tag{3}
  \sigma(B_c, B_d) = \lambda_{cd},
\end{equation}

where $\lambda_{cd}$ approximates the largest magnitude of the appropriate wave
speed of the convective operator. In spatial dimension $j$, it could be
$\lambda_{cd} = \max\{ |u_{j,c}|, |u_{j,d}| \}$.
This corresponds to a Rusanov or local Lax-Friedrichs dissipation. Other types
such as Roe dissipation could also be chosen, cf. [Svärd and Mishra (2009)](#svard2009shock).
This choice of the weights yields a shock capturing first order dissipation.

#### <a id="ex:SBP-2-SC-AD"></a>Example 2

> For the second order SBP operator given in Example 1 above, the shock
> capturing first order dissipation operator of the form (1) is
> given as in Example 1 with weights $a_1 = 0$ and $a_c$ as
> in (2), (3). For higher order operators, additional
> dissipation terms of the form $\frac{1}{2} \sigma(B_{c+2}, B_{c})$
> etc. appear.

In order to increase the order of accuracy away from discontinuities, the weights
$a$ of the artificial viscosity operators are modified adaptively. Indeed,
a limited value $\kappa_k(B_c, B_d) \sigma(B_c, B_d)$ is
used, where $\kappa_k$ takes values between zero and one, using lower and upper
cutoff coefficients $C_{\min}, C_{\max}$ as in eq. (40) of [Svärd and Mishra (2009)](#svard2009shock).
The index $k$ determines the number of grid nodes between the arguments of
$\kappa_k$ and

\begin{equation}
  \kappa_k(B_{c}, B_{d})
  =
  \kappa_k\bigl( \sigma(B_{c}, B_{d}) (B_{c} - B_{d})^2 \bigr),
\end{equation}

where

\begin{equation}
  \kappa_k(s)
  =
  \begin{cases}
    0,
    & s \leq k^2 M_{2p,\min} C_{\min} \Delta x,
    \\
    \sqrt{\frac{s - k^2 M_{2p,\min} C_{\min} \Delta x}
               {k^2 M_{2p,\max} C_{\max} \Delta x - k^2 M_{2p,\min} C_{\min} \Delta x}},
    & k^2 M_{2p,\min} C_{\min} \Delta x < s < k^2 M_{2p,\max} C_{\max} \Delta x,
    \\
    1,
    & s \geq k^2 M_{2p,\max} C_{\max} \Delta x.
  \end{cases}
\end{equation}

The term whose square root is used is a linear interpolation between zero and one.
The additional parameters $M_{2p,\min}, M_{2p,\max}$ depend on the order $2p$
of the SBP derivative operator and allow a similar choice of cutoff parameters
$C_{\min}, C_{\max}$ for schemes of different order, depending only on the special
initial boundary value problem at hand. Useful values of these parameters are
given in Table 3 of [Svärd and Mishra (2009)](#svard2009shock).

Finally, a convex combination of the shock capturing artificial dissipation with
weight $\frac{1}{2 \Delta x} \kappa_k \sigma$ and the high order artificial dissipation
with weight $1-\kappa_k$ is used.

#### <a id="ex:SBP-2-adaptive-AD"></a>Example 3

> The shock capturing artificial dissipation for the second order SBP operator
> of Example 1 is very similar to the second order artificial
> dissipation operator described there, since the applied
> differences consist only of two terms. Thus, the adaptive artificial dissipation
> is of the form (2) with weights $a_1 = 0$ and
>
> \begin{equation}
    a_{c}
    =
    \frac{1}{2 \Delta x} \kappa_1(B_c, B_{c-1}) \sigma(B_c, B_{c-1})
    + 1 - \kappa_1(B_c, B_{c-1}).
  \end{equation}
 

#### Remark

> In some plasma simulation codes, a linear filtering procedure is applied instead
> of artificial dissipation. If the dissipation weights are chosen as one and only
> interior terms are considered, such a smoothing procedure can be written as
>
> \begin{equation}
    B \mapsto \widetilde B := (1 - \alpha) B + \alpha \bar B,
    \quad
    \bar B = \frac{B_{c+1} + 2 B_c + B_{c-1}}{4}.
  \end{equation}
> 
> Thus,
>
> \begin{equation}
    \widetilde B
    =
    B + \alpha \, \Delta x^2 \, D_2 B,
    \quad
    \Delta x^2 \, D_2
    =
    \begin{pmatrix}
      \ddots & \ddots & \ddots \\
      & 1 & -2 & 1 \\
      && \ddots & \ddots & \ddots
    \end{pmatrix}.
  \end{equation}
>
> Here, $D_2$ is a discrete Laplace operator. Thus, carrying out artificial
> dissipation with constant coefficient one in a periodic domain via an operator
> splitting approach and an explicit Euler step for the diffusion part corresponds
> to this linear filtering, if the smoothing parameter $\alpha$, the step size
> $\Delta x$ and the time step $\Delta t$ satisfy $\Delta t = \alpha \Delta x^2$.
> However, since there do not seem to be significant advantages of this approach,
> the artificial dissipation without operator splitting is used.



## References

- <a id="mattsson2004stable"></a> Ken Mattsson, Magnus Svärd, Jan Nordström (2004).
  Stable and Accurate Artificial Dissipation.
  *Journal of Scientific Computing*, 21(1): 57-79.
  DOI [10.1023/B:JOMP.0000027955.75872.3f](https://dx.doi.org/10.1023/B:JOMP.0000027955.75872.3f).
- <a id="svard2009shock"></a> Magnus Svärd, Siddhartha Mishra (2009).
  Shock Capturing Artificial Dissipation for High-Order Finite Difference Schemes.
  *Journal of Scientific Computing*, 39(3): 454-484.
  DOI [10.1007/s10915-009-9285-1](https://dx.doi.org/10.1007/s10915-009-9285-1).